In [ ]:
import numpy as np
import pandas
import csv
import matplotlib.pyplot as pl
import seaborn as sns

# THIS LAST BIT IS TO NAVIAGATE TO WHERE MY CUSTOM MODULES ARE LOCATED
import os
if os.getcwd()[-4:] != 'AIMS':
    default_path = os.getcwd()[:-10]
    os.chdir(default_path)

import seq_loader
import aims_analysis as aims
import aims_classification as classy

# Apply A Pre-Trained Classifier to Test Sequences

In [ ]:
# To re-create figure 6A, we need to load in all of the naturally-derived polyreactivity data
[mono_all,poly_all,mono,poly]=seq_loader.getBunker()
[mono_jennaAll,poly_jennaAll,mono_jenna,poly_jenna]=seq_loader.getJenna()
[mono_HugoAll,poly_HugoAll,mono_Hugo,poly_Hugo]=seq_loader.getHugo()
[mono_HugNat_all,poly_HugNat_all,mono_HugNat,poly_HugNat]=seq_loader.getHugo_Nature()
[mono_PLOS,poly_PLOS]=seq_loader.getHugo_PLOS()
[mono_HugNat_allCNT,poly_HugNat_allCNT,mono_HugNatCNT,poly_HugNatCNT] = seq_loader.getHugo_NatCNTRL()

parse_mono=np.hstack((mono,mono_jenna,mono_Hugo,mono_HugNat,mono_HugNatCNT,mono_PLOS))
parse_poly=np.hstack((poly,poly_jenna,poly_Hugo,poly_HugNat,poly_HugNatCNT,poly_PLOS))

In [ ]:
# Read in your test sequences in the recommended format:
# https://aims-doc.readthedocs.io/en/latest/AIMS_basics.html#formatting

with open('/Users/alexpico/Desktop/test_seqs.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    next(reader)
    test_seqs = list(reader)

test_seqs = np.transpose(test_seqs)
test_seqs

In [ ]:
# Re-Run this command,since it is fast anyway
# Really running it here for the "MatrixSize" variable
x,y,MatrixSize = aims.gen_tcr_matrix(parse_poly,pre_mono=parse_mono,binary=True,return_Size=True)

num_mono = np.shape(parse_mono)[1]
num_poly = np.shape(parse_poly)[1]

# Pretty slow step when dealing with large matrix (full/parsed data)
numVects = 75
full_big_nat, weights_nat, acc_nat, mda_nat, parsed_mat_nat, top_names_nat = classy.do_linear_split(parse_mono,parse_poly,ridCorr = True, giveSize = MatrixSize, matSize = numVects)

# Print out the accuracy
print("Prediction accuracy: " + str(acc_nat))

# And now for the predicted likelihood of polyreactivity

In [ ]:
# Apply the pretrained LDA model to your test sequences
test_Big = classy.get_bigass_matrix(test_seqs,giveSize = MatrixSize)
test_apply = classy.apply_pretrained_LDA(test_Big,top_names_nat, weights_nat)
mda_test = test_apply
test_dim = np.shape(test_apply)[0]

print("LDA prediction values: \n" + str(mda_test))
# Note: the order of values is the same as the input order, so you can assign values to your original sequences.

In [ ]:
# A swarmplot comparing the predicted values (right) to the distribution of known entities
names = ["Naturally Derived" for x in range(num_mono+num_poly)] + ["Test Abs" for x in range(test_dim)]
full_dset = np.vstack((mda_nat,mda_test))
reacts = ["Non-Polyreactive" for x in range(num_mono)] + ["Polyreactive" for x in range(num_poly)] + ["Test" for x in range(test_dim)]

fig = pl.figure(figsize = (16, 8))
d1 = {'Dataset': names, 'Linear Discriminant 1': full_dset.reshape(len(full_dset)),
     'Reactivity' : reacts}
df1 = pandas.DataFrame(data=d1)
sns.set(style="white", color_codes=True,font_scale=1.5)
# Note, I did edit colors outside of python
zz = sns.swarmplot(x="Dataset", y="Linear Discriminant 1", data=df1, hue = 'Reactivity', palette = "Dark2")

sns.despine()
zz.legend_.remove()
pl.legend(loc='upper right')
#pl.savefig('LDA_hiv_thera.pdf',format='pdf')